In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('hackathon_sample_v2.csv')
df.head()

,date,ret_eom,permno,exchcd,shrcd,mspread,year,month,size_port,rf,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
0,20000131,20000131,10078,11,3,0.017178,2000,1,B,0.0041,...,2.219037,0.012635,0.504688,1.581540,1.373224,180,1.700939,1.711756,1.476410,1.087626
1,20000131,20000131,10104,11,3,0.019720,2000,1,B,0.0041,...,1.437330,0.016634,0.473872,1.941648,2.058353,180,0.828930,1.717670,-0.603531,0.608215
2,20000131,20000131,10107,11,3,0.011369,2000,1,B,0.0041,...,1.466883,0.003855,0.563178,1.228124,1.899772,180,1.190962,1.682187,-0.583515,1.336684
3,20000131,20000131,10108,11,1,0.010600,2000,1,B,0.0041,...,-0.285524,0.015658,0.327841,0.952188,1.187073,180,1.332815,1.453165,0.210170,1.130917
4,20000131,20000131,10119,11,1,0.003485,2000,1,B,0.0041,...,0.217867,0.004411,0.289368,0.457370,0.715285,228,0.785417,-0.381167,1.689625,0.331900


In [3]:
# Ensure the 'date' column is string (or integer will also work)
df['date'] = df['date'].astype(str)

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [4]:

start_date = '2000-01-01'
end_date = '2023-12-31'


df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

In [5]:
# Extract the year and month as a 'Year-Month' period
df_filtered['year_month'] = df_filtered['date'].dt.to_period('M')


In [6]:
monthly_stock_ticker = df_filtered.groupby(['stock_ticker', 'year_month']).size().reset_index(name='count')

monthly_stock_ticker.drop(columns=['count'], inplace=True)

full_month_range = pd.period_range(start=start_date, end=end_date, freq='M')
stock_tickers = df_filtered['stock_ticker'].unique()
full_index = pd.MultiIndex.from_product([stock_tickers, full_month_range], names=['stock_ticker', 'year_month'])

monthly_stock_ticker_complete = monthly_stock_ticker.set_index(['stock_ticker', 'year_month']).reindex(full_index, fill_value=0).reset_index()

monthly_stock_ticker_complete = monthly_stock_ticker_complete.sort_values(by=['year_month', 'stock_ticker']).reset_index(drop=True)


In [7]:
monthly_stock_ticker_complete.head()

,stock_ticker,year_month
0,A,2000-01
1,AA,2000-01
2,AAI,2000-01
3,AAL,2000-01
4,AAN,2000-01


In [8]:
monthly_stock_ticker_complete.describe()

,stock_ticker,year_month
count,1026720,1027008
unique,3565,288
top,A,2000-01
freq,288,3566


In [9]:
random_stocks = ['AAPL', 'MSFT']

df_random_selection = monthly_stock_ticker_complete[monthly_stock_ticker_complete['stock_ticker'].isin(random_stocks)]

In [10]:
cols = df_random_selection['stock_ticker'].unique().tolist()
print(cols)

['AAPL', 'MSFT']


In [11]:
df_random_selection['year_month'] = pd.to_datetime(df_random_selection['year_month'].astype(str) + '-01')  # Set to first day of month

df_random_selection['date'] = df_random_selection['year_month'] + pd.offsets.MonthEnd(0)

/var/folders/rs/ff_lqcyn0t32gsscnqlrvhbw0000gn/T/ipykernel_58334/3401991823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_random_selection['year_month'] = pd.to_datetime(df_random_selection['year_month'].astype(str) + '-01')  # Set to first day of month
/var/folders/rs/ff_lqcyn0t32gsscnqlrvhbw0000gn/T/ipykernel_58334/3401991823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_random_selection['date'] = df_random_selection['year_month'] + pd.offsets.MonthEnd(0)


In [12]:
df_random_selection = df_random_selection.drop(columns=['year_month'])
df_random_selection.head()

,stock_ticker,date
7,AAPL,2000-01-31
2112,MSFT,2000-01-31
3573,AAPL,2000-02-29
5678,MSFT,2000-02-29
7139,AAPL,2000-03-31


In [13]:
df_merged = pd.merge(df_random_selection, df_filtered, on = ['date', 'stock_ticker'])

In [14]:
df_merged.to_csv('random_ten_sample.csv', index = False)

mkt_ind.csv cleaning

In [15]:
df = pd.read_csv('mkt_ind.csv')
df.head()

,rf,sp_ret,date
0,0.0041,-0.050904,2000-01-31
1,0.0043,-0.020108,2000-02-29
2,0.0047,0.096720,2000-03-31
3,0.0046,-0.030796,2000-04-30
4,0.0050,-0.021915,2000-05-31
